In [1]:
!pip3 install --upgrade Pillow
!pip3 install tesseract-ocr
!pip3 install numpy
!pip3 install tqdm
!pip3 install os
!pip3 install codecs
!pip3 install pytesseract
!pip3 install opencv-python
!pip3 install pandas 

Requirement already up-to-date: Pillow in /usr/local/lib/python3.6/dist-packages
  Using cached https://files.pythonhosted.org/packages/e2/0d/dcee3dd0fc4c7bcd18125a98f8ba6d9db7aecaa40770595203e312649587/tesseract-ocr-0.0.1.tar.gz
  Running setup.py bdist_wheel for tesseract-ocr ... error
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-o1j4x_q6/tesseract-ocr/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpvegz_lt6pip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  file tesseract_ocr.py (for module tesseract_ocr) not found
  file tesseract_ocr.py (for module tesseract_ocr) not found
  running build_ext
  building 'tesseract_ocr' extension
  creating build
  creating build/temp.linux-x86_64-3.6
  x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-stron

In [14]:
#All required imports
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import cv2
import os
import codecs
from tqdm import tqdm
import numpy as np
import sys
import json
from decimal import *
import pandas as pd
import glob2

In [28]:
#list of the images we ar going to use 
mongroupedimages = glob2.glob('/mekeneocr/tiny/tiny_images/*.png')
mongroupedimages[:10]

['/mekeneocr/tiny/tiny_images/AAMD.png',
 '/mekeneocr/tiny/tiny_images/Aarane.png',
 '/mekeneocr/tiny/tiny_images/abacavir.png',
 '/mekeneocr/tiny/tiny_images/abae.png',
 '/mekeneocr/tiny/tiny_images/abasia.png',
 '/mekeneocr/tiny/tiny_images/abatement.png',
 '/mekeneocr/tiny/tiny_images/abiatus.png',
 '/mekeneocr/tiny/tiny_images/abidi.png',
 '/mekeneocr/tiny/tiny_images/abietine.png',
 '/mekeneocr/tiny/tiny_images/abietite.png']

In [26]:
#test on how to get the orignal (width, height) for a single image on disk
im = Image.open('/mekeneocr/tiny/tiny_images/AAMD.png')
width, height = im.size

In [27]:
im.size[1]  #get the height of AAMD.png on disk

40

In [30]:
#listing of all origanl width and height of our set of images on disk

listedeswidth = []   #create an empty list of width
listedesheight = []  #create an empty list of height
for thing in mongroupedimages:       #iterate on every single image on our set
    ima = Image.open(thing)          #open the image
    ima_width = ima.size[0]          #to this variable assign the image's width
    ima_height = ima.size[1]         #to this variable assign the image's height
    
    listedeswidth.append(ima_width)  #for every image, add to it corresponding list
    listedesheight.append(ima_height) ##for every image, add to it corresponding list

In [ ]:
listedeswidth.head()   #view the list of width

In [19]:
#load and naming the columns of the csv file genarated after the letters prediction from cmd
themeteorcsv = pd.read_csv('/mekeneocr/tiny/mycsvmeteor.csv')
themeteorcsv.columns = ['names', 'letters', 'confidence', 'x', 'y', 'w', 'h']
themeteorcsv

,names,letters,confidence,x,y,w,h
0,AAMD.png,d,56.02,20,120,18,87
1,AAMD.png,e,97.54,37,123,16,83
2,Aarane.png,s,56.63,56,129,15,57
3,Aarane.png,e,60.82,35,130,16,62
4,Aarane.png,a,71.24,35,130,16,62
...,...,...,...,...,...,...,...
310,acroagnosis.png,o,82.19,21,130,7,53
311,acroagnosis.png,o,85.84,56,129,9,54
312,acroagnosis.png,g,95.13,40,137,8,66
313,acroagnosis.png,s,96.39,71,129,7,52


In [22]:
#generate a dataframe in which we'll store some variables
pemp = themeteorcsv[["names"]].drop_duplicates().reset_index(drop = True)  

In [42]:
#how to get the full size of the AAMM.png image on disk
os.path.getsize('/mekeneocr/tiny/tiny_images/AAMD.png')  

1088

In [23]:
#this function will assign to every image its full size on disk 
def mekene(x):
    mesimages = '/mekeneocr/tiny/tiny_images'
    myfile = os.path.join(mesimages, x)
    #return os.path.getsize(myfile)
    try:
        res = os.path.getsize(myfile)
    except :
        res = 0
    return res

In [24]:
#create a new column (size), that will store the full size
pemp["size"] = pemp["names"].apply(lambda x : mekene(x))

In [25]:
pemp.head()  #view the table

,names,size
0,AAMD.png,1088
1,Aarane.png,1200
2,abacavir.png,1189
3,abae.png,906
4,abasia.png,967


In [36]:
#add to a new column the list of the orrignal width of images
pemp['image_original_width'] = listedeswidth  

In [37]:
#add to a new column the list of the orrignal height of images
pemp['image_original_height'] = listedesheight

In [40]:
pemp.head()   #view the table

,names,size,image_original_width,image_original_height
0,AAMD.png,1088,150,40
1,Aarane.png,1200,150,40
2,abacavir.png,1189,150,40
3,abae.png,906,150,40
4,abasia.png,967,150,40


In [41]:
#merge both previous table into a new one
first_whole_table = pd.merge(pemp, themeteorcsv, on = ['names'])
first_whole_table.head()

,names,size,image_original_width,image_original_height,letters,confidence,x,y,w,h
0,AAMD.png,1088,150,40,d,56.02,20,120,18,87
1,AAMD.png,1088,150,40,e,97.54,37,123,16,83
2,Aarane.png,1200,150,40,s,56.63,56,129,15,57
3,Aarane.png,1200,150,40,e,60.82,35,130,16,62
4,Aarane.png,1200,150,40,a,71.24,35,130,16,62


In [ ]:
''' NOTE: the images size on the neural network is (192, 192). And that means the (x,y,w,h) in the above table have to resize
    in order for the vgg annotator the recognise them.
    let's denote (X,Y,W,H), the resized coordinates we will use in the vgg annotator.
    The conversion shall be as follow:
    X = (x*image_original_width)/192
    Y = (y*image_original_height)/192
    W = (w*image_original_width)/192
    H = (h*image_original_height)/192   '''

In [52]:
#conversion of x to X and making it an integer
listedesX = ((first_whole_table['x'] * first_whole_table['image_original_width'])/192)
listedesX = [ int(x) for x in listedesX ]

#conversion of y to Y and making it an integer
listedesY = ((first_whole_table['y'] * first_whole_table['image_original_height'])/192)
listedesY = [ int(x) for x in listedesY ]

#conversion of w to W and making it an integer
listedesW = ((first_whole_table['w'] * first_whole_table['image_original_width'])/192)
listedesW = [ int(x) for x in listedesW ]

#conversion of h to H and making it an integer
listedesH = ((first_whole_table['h'] * first_whole_table['image_original_height'])/192)
listedesH = [ int(x) for x in listedesH ]

In [56]:
listedesH[:10]  #view the list of H

[18, 17, 11, 12, 12, 12, 16, 12, 12, 11]

In [57]:
#add the new list to the big previous table
first_whole_table['X'] = listedesX
first_whole_table['Y'] = listedesY
first_whole_table['W'] = listedesW
first_whole_table['H'] = listedesH

In [58]:
first_whole_table.head()  #view the table

,names,size,image_original_width,image_original_height,letters,confidence,x,y,w,h,X,Y,W,H
0,AAMD.png,1088,150,40,d,56.02,20,120,18,87,15,25,14,18
1,AAMD.png,1088,150,40,e,97.54,37,123,16,83,28,25,12,17
2,Aarane.png,1200,150,40,s,56.63,56,129,15,57,43,26,11,11
3,Aarane.png,1200,150,40,e,60.82,35,130,16,62,27,27,12,12
4,Aarane.png,1200,150,40,a,71.24,35,130,16,62,27,27,12,12


In [61]:
#the aim of this function is to output the inshape of the vgg annotator ({"name":"rect","x":15,"y":25,"width":14,"heigh...})

def meteor(X,Y,W,H):
    
    my_dico = {}
    my_dico["name"] ="rect"
    my_dico["x"] =X
    my_dico["y"] =Y
    my_dico["width"] =W
    my_dico["height"] =H
    
    return str(my_dico).replace("'", "\"").replace(" ","")


In [62]:
#add this new column called region_shape_attributes
first_whole_table['region_shape_attributes'] = first_whole_table.apply(lambda row : meteor(row['X'], 
                     row['Y'], row['W'], row['H']), axis = 1) 

In [63]:
first_whole_table.head() #view the table

,names,size,image_original_width,image_original_height,letters,confidence,x,y,w,h,X,Y,W,H,region_shape_attributes
0,AAMD.png,1088,150,40,d,56.02,20,120,18,87,15,25,14,18,"{""name"":""rect"",""x"":15,""y"":25,""width"":14,""heigh..."
1,AAMD.png,1088,150,40,e,97.54,37,123,16,83,28,25,12,17,"{""name"":""rect"",""x"":28,""y"":25,""width"":12,""heigh..."
2,Aarane.png,1200,150,40,s,56.63,56,129,15,57,43,26,11,11,"{""name"":""rect"",""x"":43,""y"":26,""width"":11,""heigh..."
3,Aarane.png,1200,150,40,e,60.82,35,130,16,62,27,27,12,12,"{""name"":""rect"",""x"":27,""y"":27,""width"":12,""heigh..."
4,Aarane.png,1200,150,40,a,71.24,35,130,16,62,27,27,12,12,"{""name"":""rect"",""x"":27,""y"":27,""width"":12,""heigh..."


In [64]:
#the aim of this function is to output the inshape of the vgg annotator ({"name":"d"})

def meteor2(x):
    
    my_dico2 = {}
    my_dico2["name"] =x
   
    
    return str(my_dico2).replace("'", "\"").replace(" ","")
    

In [65]:
#add this new column called region_attributes
first_whole_table['region_attributes'] = first_whole_table.apply(lambda row : meteor2(row['letters']), axis = 1) 

In [66]:
first_whole_table.head()  #view the table

,names,size,image_original_width,image_original_height,letters,confidence,x,y,w,h,X,Y,W,H,region_shape_attributes,region_attributes
0,AAMD.png,1088,150,40,d,56.02,20,120,18,87,15,25,14,18,"{""name"":""rect"",""x"":15,""y"":25,""width"":14,""heigh...","{""name"":""d""}"
1,AAMD.png,1088,150,40,e,97.54,37,123,16,83,28,25,12,17,"{""name"":""rect"",""x"":28,""y"":25,""width"":12,""heigh...","{""name"":""e""}"
2,Aarane.png,1200,150,40,s,56.63,56,129,15,57,43,26,11,11,"{""name"":""rect"",""x"":43,""y"":26,""width"":11,""heigh...","{""name"":""s""}"
3,Aarane.png,1200,150,40,e,60.82,35,130,16,62,27,27,12,12,"{""name"":""rect"",""x"":27,""y"":27,""width"":12,""heigh...","{""name"":""e""}"
4,Aarane.png,1200,150,40,a,71.24,35,130,16,62,27,27,12,12,"{""name"":""rect"",""x"":27,""y"":27,""width"":12,""heigh...","{""name"":""a""}"


In [70]:
groupes = first_whole_table.groupby('names')
lesbonsgroupes = first_whole_table.groupby('names').agg('count')
groupement = groupes.groups
print(groupement)
#print(lesbonsgroupes)

{'AAMD.png': [0, 1], 'ABVD.png': [92], 'ACC.png': [118, 119], 'ACEP.png': [140], 'ACIS.png': [274, 275, 276], 'ACR.png': [283, 284], 'Aarane.png': [2, 3, 4, 5, 6, 7], 'Abiomed.png': [56, 57, 58, 59], 'Absinthism.png': [67, 68, 69, 70, 71, 72], 'Achromycin.png': [214, 215, 216, 217, 218, 219, 220], 'Achucarros.png': [221, 222, 223, 224, 225], 'Acidaminococcus.png': [243, 244, 245, 246, 247, 248], 'abacavir.png': [8, 9, 10, 11, 12, 13], 'abae.png': [14, 15], 'abasia.png': [16, 17, 18, 19, 20], 'abatement.png': [21, 22, 23, 24, 25, 26, 27, 28], 'abiatus.png': [29, 30, 31, 32, 33], 'abidi.png': [34, 35, 36, 37], 'abietine.png': [38, 39, 40, 41, 42, 43], 'abietite.png': [44, 45, 46, 47, 48], 'abiogenous.png': [49, 50, 51, 52, 53, 54, 55], 'ablastin.png': [60, 61, 62, 63, 64, 65, 66], 'absorptiometer.png': [73, 74, 75, 76, 77, 78, 79, 80, 81], 'abstinence.png': [82, 83, 84, 85, 86], 'abuttals.png': [87, 88, 89, 90, 91], 'aca.png': [93, 94], 'acalephan.png': [95, 96, 97, 98, 99, 100, 101, 102

In [ ]:
temporary_dataframe = lesbonsgroupes[['size']]
temporary_dataframe.columns = ['mycounts']
temporary_dataframe.reset_index(inplace = True)
temporary_dataframe.head()